In [1]:
import open3d as open3d 
from visualization_utils import * 
import numpy as np 
from metrics_class import Metrics

In [2]:
out_folder = 'pcd_preprocessed/'

pcd_gt = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti_instance.pcd')
pred_pcd = o3d.io.read_point_cloud(out_folder + 'ncuts_instances_tarl_spatial1.pcd')
dbscan_pcd = o3d.io.read_point_cloud(out_folder + 'dbscan_instances.pcd')


unique_colors, labels_ncuts = np.unique(np.asarray(pred_pcd.colors), axis=0, return_inverse=True)
unique_colors, labels_dbscan = np.unique(np.asarray(dbscan_pcd.colors), axis=0, return_inverse=True)
unique_colors, labels_kitti = np.unique(np.asarray(pcd_gt.colors),axis=0, return_inverse=True)




In [3]:
metrics_db = Metrics('dbscan')
metrics_pred = Metrics('tarl spatial 2')
#metrics_db.calc_ap = False ##AP was already calculated before 
metrics_pred.calc_ap = False

#metrics_db.update_stats(labels_dbscan,labels_kitti)
metrics_pred.update_stats(labels_ncuts,labels_kitti)



{'panoptic': 0.6617402324115282, 'precision': 0.8023255813953488, 'recall': 0.6448598130841121, 'fScore': 0.7150259067357513, 'usr': 0.11627906976744186, 'osr': 0.009345794392523364, 'noise': 0.11627906976744186, 'missed': 0.19626168224299065, 'mean': 0.9254772815610504}
P/R Metrics for dbscan
Precision  80.23
Recall  64.49
F1Score 71.5
